In [1]:
import time
import numpy as np
from tqdm import tqdm
from class_data import *

In [2]:
path_data = '/Users/federicogallina/Documents/PhD documents/Data/'
baci_documentation = Path(path_data + 'BACI_documentation')
baci_data = Path(path_data + 'trade_flows/BACI_HS07_V202001')
path_temp = Path('temp/')
plots_data = Path('data/')

In [3]:
country_codes = pd.read_csv(baci_documentation / "country_codes_V202001.csv", encoding='latin-1', dtype = {'country_code': 'str', 'country_name_abbreviation': 'str', 'country_name_full': 'str', 'iso_2digit_alpha': 'str', 'iso_3digit_alpha': 'str'})

In [4]:
full_kor = pd.DataFrame()
def cleaning_baci(year, country_df, empty_df, substring, col_origin, id_origin, col_destination, id_destination, input_path, output_path, root_output_file):
    df = pd.read_csv(input_path / f"BACI_HS07_Y{year}_V202001.csv")
    baci = BACI(df, country_df)
    baci.adjust_columns()
    baci.match_country_codes('origin')
    baci.match_country_codes('destination')
    baci.slice_countries(substring, {col_origin: id_origin, col_destination: id_destination})
    baci.trade_flow_class({col_origin: id_origin, col_destination: id_destination})
    baci.industry_classification()
    baci.data.to_csv(output_path / f"{root_output_file}_{year}.csv", index = False)
    empty_df = empty_df.append(baci.data, ignore_index = True)
    return empty_df

In [5]:

start = time.time()
with tqdm(total=120) as pbar:
    for year in range(2007, 2019):
        full_kor = cleaning_baci(year, country_codes, full_kor, 'or', 'origin_iso3', 'KOR', 'destination_iso3', 'KOR', baci_data, path_temp, 'kor')
        pbar.update(10)
end = time.time()
print(end - start)

100%|██████████| 120/120 [12:48<00:00,  6.41s/it]768.6387960910797



In [6]:
baci_full = BACI(full_kor, country_codes)
baci_full.total_flow()
baci_full.aggregate_country(BACI.eu28, "EU28", "0")
baci_full.aggregate_country(BACI.eu27, "EU27", "1")
export_df = baci_full.data
export_df = export_df[export_df['trade_flow'] == "Import"]
export_df['value_log'] = np.log(export_df['value'])
export_df['value_change'] = export_df.groupby(['origin_name', 'destination_name', 'product_code'], as_index = False)['value'].diff()
export_df['value_growth'] = export_df['value_change']/(export_df['value'] - export_df['value_change'])
export_df.to_csv(path_temp / "kor_edited.csv", index = False)

In [7]:
with tqdm(total=120) as pbar:
    for year in list(export_df['year'].unique()):
        temp_df = export_df[export_df['year'] == year].copy()
        temp_df.drop(columns = ['quantity', 'origin_iso3', 'destination_iso3', 'chapters', 'industry'], inplace = True)
        temp_df.to_csv(plots_data / f'slice_{year}.csv', index = False)
        pbar.update(10)

100%|██████████| 120/120 [00:09<00:00, 12.10it/s]
